### This file sets up the directories, imports py libs, runs push_and_build.sh, starts the session, does training and deployment

In [1]:
!mkdir opt
!mkdir opt/ml
!mkdir opt/ml/input
!mkdir opt/ml/input/data
!mkdir opt/ml/input/data/training
!mkdir opt/ml/model
!mkdir opt/ml/output
!mkdir opt/program

mkdir: cannot create directory ‘opt’: File exists
mkdir: cannot create directory ‘opt/ml’: File exists
mkdir: cannot create directory ‘opt/ml/input’: File exists
mkdir: cannot create directory ‘opt/ml/input/data’: File exists
mkdir: cannot create directory ‘opt/ml/input/data/training’: File exists
mkdir: cannot create directory ‘opt/ml/model’: File exists
mkdir: cannot create directory ‘opt/ml/output’: File exists
mkdir: cannot create directory ‘opt/program’: File exists


In [11]:
#!mkdir packages
#os.chdir('packages')
#!pip install tensorflow -t ./
#!pip install keras
#!pip install gunicorn  

In [23]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
import re
import boto3
import json


import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role
import os


In [24]:
os.getcwd()


'/home/ec2-user/SageMaker/test_mlops223/container'

In [16]:
os.chdir('/home/ec2-user/SageMaker/test_mlops223/container')

In [45]:
! ./build_and_push.sh sagemaker-lstm

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  610.6MB
Step 1/8 : FROM python:3.6
 ---> 1daf62e8cab5
Step 2/8 : COPY "requirements.txt" .
 ---> Using cache
 ---> d5a444b08f93
Step 3/8 : RUN ["pip", "install","-r", "requirements.txt"]
 ---> Using cache
 ---> 21b42fab03ad
Step 4/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> c05361c0b37d
Step 5/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> f9737a0acf0b
Step 6/8 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 8ecd488c00c8
Step 7/8 : COPY algorithms /opt/program
 ---> Using cache
 ---> 91bf5a282a94
Step 8/8 : WORKDIR /opt/program
 ---> Using cache
 ---> eb4c59d11d50
Successfully built eb4c59d11d50
Successf

In [46]:
role = get_execution_role()
sess = sage.Session()
bucket = "sagemaker-wisdm2" #from get_data.ipynb
data_location = 's3://{}/data'.format(bucket)


In [47]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-lstm:latest'.format(account, region)


In [48]:
lstm = sage.estimator.Estimator(image,
                       role, 1, 'ml.m5.4xlarge',
                       output_path="s3://{}/output".format(bucket),
                       sagemaker_session=sess)

In [49]:
lstm.fit(data_location)

2020-03-26 17:01:05 Starting - Starting the training job...
2020-03-26 17:01:07 Starting - Launching requested ML instances.........
2020-03-26 17:02:41 Starting - Preparing the instances for training...
2020-03-26 17:03:31 Downloading - Downloading input data...
2020-03-26 17:03:45 Training - Downloading the training image.......Using TensorFlow backend.
2020-03-26 17:05:09.788718: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-26 17:05:09.788814: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-26 17:05:09.788826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, pl

In [60]:
#finally we export the parameters we need for deployment as json and upload it to s3
outjson = {'jobTrainingId': lstm._current_job_name, 'outputPath': "s3://{}/output".format(bucket), 'imageLocation':image}

with open("hyperParameters.json", "w") as json_file:
    json.dump(outjson, json_file)
    
    

In [63]:
bucket = "sagemaker-wisdm2"
data_location = 's3://{}'.format(bucket)

s3 = boto3.resource('s3')

s3.Bucket(bucket).upload_file("hyperParameters.json","hyperParameters.json" )

